In [1]:
import os
import pandas as pd
import hopsworks
import joblib
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from PIL import Image
from dotenv import load_dotenv
from src.paths import PARENT_DIR

c:\Users\vivek\Documents\iris\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
HOPSWORKS_PROJECT_NAME = 'Vivekmaj1'

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/552084
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_view = fs.get_feature_view(name="iris", version=1)

In [4]:
class ExtractExtraFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Create new features
        X['sepal_area'] = X['sepal_length'] * X['sepal_width']
        X['petal_area'] = X['petal_length'] * X['petal_width']
        X['sepal_petal_length_diff'] = X['sepal_length'] - X['petal_length']
        X['sepal_petal_width_diff'] = X['sepal_width'] - X['petal_width']
        X['sepal_length_to_width'] = X['sepal_length'] / X['sepal_width']
        X['petal_length_to_width'] = X['petal_length'] / X['petal_width']
        
        return X

In [5]:
mr = project.get_model_registry()
model = mr.get_model("iris_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/iris_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
batch_data = feature_view.get_batch_data()

y_pred = model.predict(batch_data)

y_pred

Finished: Reading data from Hopsworks, using ArrowFlight (0.77s) 


array(['Setosa', 'Versicolor', 'Virginica', 'Versicolor', 'Setosa',
       'Versicolor', 'Virginica', 'Versicolor', 'Setosa', 'Setosa',
       'Setosa', 'Setosa', 'Virginica', 'Virginica', 'Setosa',
       'Virginica', 'Versicolor', 'Setosa', 'Versicolor', 'Versicolor',
       'Setosa', 'Versicolor', 'Setosa', 'Setosa', 'Versicolor',
       'Versicolor', 'Virginica', 'Virginica', 'Virginica', 'Versicolor',
       'Versicolor', 'Setosa', 'Setosa', 'Virginica', 'Setosa',
       'Virginica', 'Setosa', 'Versicolor', 'Virginica', 'Versicolor',
       'Virginica', 'Setosa', 'Versicolor', 'Virginica', 'Versicolor',
       'Setosa', 'Versicolor', 'Versicolor', 'Virginica', 'Setosa',
       'Setosa', 'Virginica', 'Virginica', 'Versicolor', 'Versicolor',
       'Virginica', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa',
       'Virginica', 'Virginica', 'Virginica', 'Setosa', 'Virginica',
       'Setosa', 'Versicolor', 'Versicolor', 'Setosa', 'Setosa', 'Setosa',
       'Virginica', 'Versicol

In [7]:
flower = y_pred[y_pred.size-1]
flower_img = "../assets/" + flower + ".png"
img = Image.open(flower_img)            

img.save("../assets/latest_iris.png")

In [8]:
iris_fg = fs.get_feature_group(name="iris", version=1)
df = iris_fg.read()
df

Finished: Reading data from Hopsworks, using ArrowFlight (0.68s) 


,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.700000,3.800000,1.700000,0.300000,Setosa
1,5.600000,2.700000,4.200000,1.300000,Versicolor
2,6.300000,3.300000,6.000000,2.500000,Virginica
3,6.000000,2.900000,4.500000,1.500000,Versicolor
4,5.000000,3.000000,1.600000,0.200000,Setosa
...,...,...,...,...,...
145,6.200000,2.800000,4.800000,1.800000,Virginica
146,6.500000,3.000000,5.500000,1.800000,Virginica
147,6.400000,2.800000,5.600000,2.100000,Virginica
148,5.000000,3.400000,1.600000,0.400000,Setosa


In [9]:
label = df.iloc[-1]["variety"]
label

'Virginica'

In [10]:
label_flower = "../assets/" + label + ".png"

img = Image.open(label_flower)            

img.save("../assets/actual_iris.png")

In [11]:
monitor_fg = fs.get_or_create_feature_group(name="iris_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Iris flower Prediction/Outcome Monitoring"
                                 )

In [16]:
print(label)

Virginica


In [17]:
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': [flower],
    'label': [label],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:04 | Remaining Time: 00:00

Launching job: iris_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/552084/jobs/named/iris_predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x120729b6af0>, None)

In [18]:
history_df = monitor_fg.read()
history_df

Finished: Reading data from Hopsworks, using ArrowFlight (0.65s) 


,prediction,label,datetime
0,Setosa,Setosa,"03/12/2024, 17:39:29"
1,Setosa,Setosa,"03/12/2024, 17:41:01"
2,Setosa,Virginica,"03/12/2024, 17:59:46"
